<a href="https://colab.research.google.com/github/shoaibanwar13/Developer_Hub_Internship_tasks/blob/master/week_2_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#Load all necessary libraries
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score





In [23]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
df=pd.read_csv('/content/spam_ham_dataset.csv')

In [25]:
print(df.isnull().sum())
df
#check for null values

Unnamed: 0    0
label         0
text          0
label_num     0
dtype: int64


,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


**Step 1: Gather Spam Emails in  Dataset**

In [27]:
def process_email(email):
  email=email.lower() #convert into lowercase
  email=re.sub(r'\d+','',email) #remove numbers
  email=re.sub(r'\w+','',email) #remove remove special character
  words=word_tokenize(email)
  stop_words=set(stopwords.words('english'))
  filtered_words=[words for words in words if words not in stop_words] # Remove stopwords
  return filtered_words

Step 2: Preprocess Emails
Preprocessing is essential to remove irrelevant characters, convert all text to lowercase, and tokenize the emails.

Convert text to lowercase to ensure uniformity.
Remove stop words (e.g., "the", "is", "at") and special characters (e.g., punctuation).
Tokenize the text into words (tokens).
Stemming/Lemmatization can be applied to reduce words to their root form.

In [28]:

vectorizer=CountVectorizer()
x=vectorizer.fit_transform(df['text'])
print("Feature name words:")
print(vectorizer.get_feature_names_out())
print("Word frequency matrix")
print(x.toarray())

df_bow=pd.DataFrame(x.toarray(),columns=vectorizer.get_feature_names_out())
print(df_bow)


Feature name words:
['00' '000' '0000' ... 'zzocb' 'zzso' 'zzsyt']
Word frequency matrix
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
      00  000  0000  000000  000000000002858  000000000049773  000080  000099  \
0      1    0     0       0                0                0       0       0   
1      0    0     0       0                0                0       0       0   
2      0    0     0       0                0                0       0       0   
3      0    0     0       0                0                0       0       0   
4      0    0     0       0                0                0       0       0   
...   ..  ...   ...     ...              ...              ...     ...     ...   
5166   0    0     0       0                0                0       0       0   
5167   0    4     0       0                0                0       0       0   
5168   0    0     0       0                0                0 

In [29]:
specific_words = ['free', 'win', 'click', 'urgent']
df_specific_words = df_bow[specific_words]
print(df_specific_words)


      free  win  click  urgent
0        0    0      0       0
1        0    0      0       0
2        0    0      0       0
3        0    0      0       0
4        0    0      0       0
...    ...  ...    ...     ...
5166     0    0      0       0
5167     0    0      0       0
5168     0    0      0       0
5169     0    0      0       0
5170     0    0      1       0

[5171 rows x 4 columns]


Step 3: Apply Bag of Words (BoW)
Bag of Words represents the frequency of each word in the email content. You can use CountVectorizer from sklearn to transform the emails into vectors.To analyze frequently used words in spam emails, you can visualize the top N frequent words using the BoW model. You can also filter words like “free,” “win,” “click,” or “urgent” to evaluate their occurrence specifically in spam.

In [30]:


#  email labels (1: Spam, 0: Not Spam)
labels = df['label']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(x, labels, test_size=0.2, random_state=42)

# Train Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Predict on test data
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.978743961352657


We train the model naive_bayes for prediction

In [31]:
df['text'].iloc[0]

"Subject: enron methanol ; meter # : 988291\r\nthis is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary\r\nflow data provided by daren } .\r\nplease override pop ' s daily volume { presently zero } to reflect daily\r\nactivity you can obtain from gas control .\r\nthis change is needed asap for economics purposes ."

In [33]:
email_index_10 = df['text'].iloc[10]  # Get  email text at index 10

#  Transform the email using the vectorizer (same vectorizer used during training)
email_index_10_transformed = vectorizer.transform([email_index_10])

# Predict using the trained classifier
prediction = clf.predict(email_index_10_transformed)

# Step 4: Output the prediction result
print(f"The email at index 10 is classified as: {'Spam' if prediction[0] == 1 else 'Not Spam'}")

The email at index 10 is classified as: Not Spam


Randomaly we chaeck email is its spam or not